In [2]:
# %% Imports
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import copy
import pandas as pd

ModuleNotFoundError: No module named 'torch'

In [3]:
# Set GPU device
print(torch.cuda.is_available())
device = torch.device("mps")


# %% Load data
TRAIN_ROOT = "data/brain_mri/training"
TEST_ROOT = "data/brain_mri/testing"
train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_ROOT)
test_dataset = torchvision.datasets.ImageFolder(root=TRAIN_ROOT)

NameError: name 'torch' is not defined

In [3]:
# %% Building the model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.vgg16 = models.vgg16(weights=True)

        # Replace output layer according to our problem
        in_feats = self.vgg16.classifier[6].in_features
        self.vgg16.classifier[6] = nn.Linear(in_feats, 4)

    def forward(self, x):
        x = self.vgg16(x)
        return x

model = CNNModel()
model.to(device)
model

/opt/homebrew/Caskroom/miniforge/base/envs/ip4rs/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CNNModel(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [4]:
# %% Prepare data for pretrained model
train_dataset = torchvision.datasets.ImageFolder(
        root=TRAIN_ROOT,
        transform=transforms.Compose([
                      transforms.Resize((255,255)),
                      transforms.ToTensor()
        ])
)

test_dataset = torchvision.datasets.ImageFolder(
        root=TEST_ROOT,
        transform=transforms.Compose([
                      transforms.Resize((255,255)),
                      transforms.ToTensor()
        ])
)

In [5]:
# %% Create data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [6]:
# %% Train
cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
epochs = 10

# Iterate x epochs over the train data
for epoch in range(epochs):
    for i, batch in enumerate(train_loader, 0):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        # Labels are automatically one-hot-encoded
        loss = cross_entropy_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        print(loss)

/opt/homebrew/Caskroom/miniforge/base/envs/ip4rs/lib/python3.9/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525849783/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


tensor(1.3803, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3702, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.5514, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3418, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.2982, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3281, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3232, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.2952, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3005, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3619, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3008, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.2352, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.1262, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.1534, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3038, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.1140, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.1901, device='mps:0', grad_fn=<NllLossBackward0

tensor(0.4104, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4353, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3657, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3346, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3898, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4546, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3051, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4818, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2467, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4799, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2693, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2609, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3008, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2797, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2733, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4600, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3400, device='mps:0', grad_fn=<NllLossBackward0

tensor(0.1615, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1341, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2897, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1291, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1534, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1569, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0957, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1082, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2724, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2540, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1364, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2960, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1786, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1195, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0679, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0546, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2297, device='mps:0', grad_fn=<NllLossBackward0

tensor(0.0315, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0372, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1218, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0245, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0293, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0626, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0642, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0607, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0520, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0930, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0603, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0724, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0262, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1661, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0291, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0313, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0895, device='mps:0', grad_fn=<NllLossBackward0

tensor(0.0128, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0390, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0258, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0311, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0126, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0509, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0735, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0068, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0265, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0141, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0583, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0073, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0224, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0534, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0061, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0445, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0228, device='mps:0', grad_fn=<NllLossBackward0

tensor(0.0020, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0062, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0062, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0166, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0195, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0016, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0289, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0287, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.1434, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0266, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0015, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0042, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0021, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0143, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0275, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0485, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0035, device='mps:0', grad_fn=<NllLossBackward0

tensor(0.0450, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0752, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0011, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0027, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0148, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0510, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0118, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0026, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0014, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0046, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0105, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0077, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0014, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0007, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0010, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0027, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.0019, device='mps:0', grad_fn=<NllLossBackward0

In [7]:
# %% Inspect predictions for first batch
import pandas as pd
inputs, labels = next(iter(test_loader))
inputs = inputs.to(device)
labels = labels.numpy()
outputs = model(inputs).max(1).indices.detach().cpu().numpy()
comparison = pd.DataFrame()
print("Batch accuracy: ", (labels==outputs).sum()/len(labels))
comparison["labels"] = labels

comparison["outputs"] = outputs
comparison

Batch accuracy:  0.78125


,labels,outputs
0,1,1
1,0,1
2,1,1
3,1,1
4,0,0
5,1,1
6,3,1
7,2,2
8,1,1
9,3,3


In [8]:
def new_layer(layer, g):
    """Clone a layer and pass its parameters through the function g."""
    layer = copy.deepcopy(layer)
    try: layer.weight = torch.nn.Parameter(g(layer.weight))
    except AttributeError: pass
    try: layer.bias = torch.nn.Parameter(g(layer.bias))
    except AttributeError: pass
    return layer

def dense_to_conv(layers):
    """ Converts a dense layer to a conv layer """
    newlayers = []
    for i,layer in enumerate(layers):
        if isinstance(layer, nn.Linear):
            newlayer = None
            if i == 0:
                m, n = 512, layer.weight.shape[0]
                newlayer = nn.Conv2d(m,n,7)
                newlayer.weight = nn.Parameter(layer.weight.reshape(n,m,7,7))
            else:
                m,n = layer.weight.shape[1],layer.weight.shape[0]
                newlayer = nn.Conv2d(m,n,1)
                newlayer.weight = nn.Parameter(layer.weight.reshape(n,m,1,1))
            newlayer.bias = nn.Parameter(layer.bias)
            newlayers += [newlayer]
        else:
            newlayers += [layer]
    return newlayers

def get_linear_layer_indices(model):
    offset = len(model.vgg16._modules['features']) + 1
    indices = []
    for i, layer in enumerate(model.vgg16._modules['classifier']):
        if isinstance(layer, nn.Linear):
            indices.append(i)
    indices = [offset + val for val in indices]
    return indices

In [9]:
def apply_lrp_on_vgg16(model, image):
    image = torch.unsqueeze(image, 0)
    # >>> Step 1: Extract layers
    layers = list(model.vgg16._modules['features']) \
                + [model.vgg16._modules['avgpool']] \
                + dense_to_conv(list(model.vgg16._modules['classifier']))
    linear_layer_indices = get_linear_layer_indices(model)
    # >>> Step 2: Propagate image through layers and store activations
    n_layers = len(layers)
    activations = [image] + [None] * n_layers # list of activations

    for layer in range(n_layers):
        if layer in linear_layer_indices:
            if layer == 32:
                activations[layer] = activations[layer].reshape((1, 512, 7, 7))
        activation = layers[layer].forward(activations[layer])
        if isinstance(layers[layer], torch.nn.modules.pooling.AdaptiveAvgPool2d):
            activation = torch.flatten(activation, start_dim=1)
        activations[layer+1] = activation

    # >>> Step 3: Replace last layer with one-hot-encoding
    output_activation = activations[-1].detach().cpu().numpy()
    max_activation = output_activation.max()
    one_hot_output = [val if val == max_activation else 0
                        for val in output_activation[0]]

    activations[-1] = torch.FloatTensor([one_hot_output]).to(device)

    # >>> Step 4: Backpropagate relevance scores
    relevances = [None] * n_layers + [activations[-1]]
    # Iterate over the layers in reverse order
    for layer in range(0, n_layers)[::-1]:
        current = layers[layer]
        # Treat max pooling layers as avg pooling
        if isinstance(current, torch.nn.MaxPool2d):
            layers[layer] = torch.nn.AvgPool2d(2)
            current = layers[layer]
        if isinstance(current, torch.nn.Conv2d) or \
           isinstance(current, torch.nn.AvgPool2d) or\
           isinstance(current, torch.nn.Linear):
            activations[layer] = activations[layer].data.requires_grad_(True)

            # Apply variants of LRP depending on the depth
            # see: https://link.springer.com/chapter/10.1007%2F978-3-030-28954-6_10
            # Lower layers, LRP-gamma >> Favor positive contributions (activations)
            if layer <= 16:       rho = lambda p: p + 0.25*p.clamp(min=0); incr = lambda z: z+1e-9
            # Middle layers, LRP-epsilon >> Remove some noise / Only most salient factors survive
            if 17 <= layer <= 30: rho = lambda p: p;                       incr = lambda z: z+1e-9+0.25*((z**2).mean()**.5).data
            # Upper Layers, LRP-0 >> Basic rule
            if layer >= 31:       rho = lambda p: p;                       incr = lambda z: z+1e-9

            # Transform weights of layer and execute forward pass
            z = incr(new_layer(layers[layer],rho).forward(activations[layer]))
            # Element-wise division between relevance of the next layer and z
            s = (relevances[layer+1]/z).data
            # Calculate the gradient and multiply it by the activation
            (z * s).sum().backward();
            c = activations[layer].grad
            # Assign new relevance values
            relevances[layer] = (activations[layer]*c).data
        else:
            relevances[layer] = relevances[layer+1]

    # >>> Potential Step 5: Apply different propagation rule for pixels
    return relevances[0]

In [ ]:
# Calculate relevances for first image in this test batch
image_id = 0
image_relevances = apply_lrp_on_vgg16(model, inputs[image_id])
image_relevances = image_relevances.permute(0,2,3,1).detach().cpu().numpy()[0]
image_relevances = np.interp(image_relevances, (image_relevances.min(),
                                                image_relevances.max()),
                                                (0, 1))
# Show relevances
truth_label = list(test_dataset.class_to_idx.keys())[
             list(test_dataset.class_to_idx.values())
            .index(labels[image_id])]

pred_label = list(test_dataset.class_to_idx.keys())[
             list(test_dataset.class_to_idx.values())
            .index(outputs[image_id])]


#print("Groundtruth for this image: ", pred_label)

# Plot images next to each other
fig, axs = plt.subplots(1,2, constrained_layout=True)


axs[0].imshow(inputs[image_id].permute(1,2,0).detach().cpu().numpy())
axs[0].axis('off')
axs[0].set_xlabel(f"Groundtruth: {truth_label}")
axs[0].set_title('Input')

fig.gca().set_yticklabels([])
fig.gca().set_xticklabels([])
fig.gca().set_xticks([])
fig.gca().set_yticks([])

axs[1].imshow(image_relevances[:,:,0], cmap="seismic")
#axs[1].axis('off')
axs[1].set_xlabel(f"Prediction: {pred_label}")
axs[1].set_title('Relevance of input features with LRP')



plt.show()


NameError: name 'apply_lrp_on_vgg16' is not defined